In [1]:
import pandas as pd
import numba
import numpy as np
from scipy.stats import zscore
import pandas_datareader as web
import os
import pickle
from constants import DATA_PATH, OUTPUT_PATH

In [2]:
df = pd.read_csv(os.path.join(DATA_PATH, 'data.csv')).set_index(['equity', 'date'])

px_tmp = df['PX_LAST'].values
df = df.groupby(level=0).shift(3)
df['PX_LAST'] = px_tmp

In [3]:
rfr = web.get_data_fred("IRLTLT01EZM156N", df.index.get_level_values('date').min(), df.index.get_level_values('date').max()).div(100)
rfr = pd.Series(index=df.index.get_level_values('date').unique()[1:], data=rfr.values[:, 0], name='risk_free_rate')

In [4]:
df_prices = df['PX_LAST'].unstack(level=0).replace(0, np.nan)
df_log_rtn = df_prices.apply(lambda x: np.log(x / x.shift(1))).dropna(how='all')
benchmark_rtn = df_log_rtn.mean(skipna=True, axis='columns')

In [5]:
def select_best_equities_from_factor(df_full, df_rnt, factor_selected, num_equities):
    factor_values = df_full[factor_selected].unstack(level=0).iloc[1:]
    equities =  factor_values.apply(
        lambda fact_val: fact_val.loc[~df_rnt.loc[fact_val.name, fact_val.index].isna()
        ].sort_values(ascending=False).head(num_equities).index.values,
        axis='columns', result_type='expand')
    equities.columns = [str(i) + "_name" for i in range(len(equities.columns))]
    return equities, factor_values

def compute_return_from_equities(df_rtn, equities):
    equities_rtn = equities.apply(lambda row: df_rtn.loc[row.name, row.values].values, axis='columns', result_type='expand')
    equities_rtn.columns = [str(i) + "_rtn" for i in range(len(equities_rtn.columns))]
    equities_rtn.columns.name = 'equity_rtn'
    return equities_rtn

def extract_portfolio_state(equities, factor_values):
    equities_sparse = pd.DataFrame(index=factor_values.index, columns=factor_values.columns, data=0)
    for date in equities_sparse.index:
        equities_sparse.loc[date, equities.loc[date].values] = 1
    equities_sparse = (2 * equities_sparse - equities_sparse.shift(1, fill_value=0)).dropna()
    return equities_sparse.applymap(lambda x: 0.33 if x == 0 else 0.66 if x == 1 else 1 if x == 2 else 0)

def apply_fees(equities_rtn, equities_names, port_state, fees):
    return equities_rtn.apply(
        lambda rtn_row: rtn_row -
                        (fees * (port_state.loc[rtn_row.name, equities_names.loc[rtn_row.name]] == 1)
                         * (1 + rtn_row).values).values,
        axis='columns')

def compute_univariate_strategy(df_full, df_rtn, factor_selected, num_equities, market_rtn, fees=None):
    equities_names, factor_values = select_best_equities_from_factor(df_full, df_rtn, factor_selected, num_equities)
    equities_rtn = compute_return_from_equities(df_rtn, equities_names)
    port_state = extract_portfolio_state(equities_names, factor_values)

    if fees is not None:
        equities_rtn = apply_fees(equities_rtn, equities_names, port_state, fees)

    equities = equities_names.merge(equities_rtn, left_index=True, right_index=True)
    equities = equities.iloc[:, np.concatenate([[i, i + num_equities] for i in range(num_equities)])]
    equities.columns = pd.MultiIndex.from_product([range(num_equities), ['name', 'rtn']])
    equities.columns.names = ['equity_pos', 'info']

    ris = equities.apply(lambda row: np.average(row.loc[:, "rtn"].values), axis='columns').to_frame(name='strategy_rtn')
    ris['strategy_alpha'] = ris['strategy_rtn'] - market_rtn
    info_ratio = ris['strategy_alpha'].mean() / ris['strategy_alpha'].std()
    return equities, ris, info_ratio, port_state

In [6]:
window_date_size = 12

@numba.jit(nopython=True)
def compute_window_corr_avg(window):
    if len(window) == window_date_size:
        corr_matrix = np.corrcoef(window, rowvar=False)
        corr_matrix_nod_iag = np.extract(~np.eye(corr_matrix.shape[0], dtype=numba.boolean), corr_matrix).reshape(corr_matrix.shape[0], -1)
        avg_corr = np.sum(corr_matrix_nod_iag, axis=1) / corr_matrix_nod_iag.shape[1]

    else:
        avg_corr = np.ones(window.shape[1])
    return avg_corr

In [7]:
def compute_zscore_strategy_simple(df_full, df_rtn, factors_selected, num_equities, market_rtn, fees=None):
    df_full['zscore'] = df_full[factors_selected].groupby(level=0).apply(lambda x: zscore(x, nan_policy='omit').mean(axis=1)).values
    return compute_univariate_strategy(df_full, df_rtn, 'zscore', num_equities, market_rtn, fees)

def compute_zscore_strategy_weighted(df_full, df_rtn, factors_selected, num_equities, market_rtn, fees=None):
    zs_factors = df[factors_selected].groupby(level=0, group_keys=False).apply(lambda x: zscore(x, nan_policy='omit'))

    zs_weight = zs_factors.groupby(level=0).rolling(window_date_size, min_periods=1, method="table"
                                                    ).apply(compute_window_corr_avg, raw=True, engine="numba").values

    zs_weight = 1 - abs(zs_weight) + 0.000001 # si somma un piccolo valore per le prime date in cui i pesi sono 0
    df_full['zscore'] = np.average(zs_factors.values, axis=1, weights=zs_weight)
    return compute_univariate_strategy(df_full, df_rtn, 'zscore', num_equities, market_rtn, fees)

In [8]:
def compute_sequential_screening(df_full, df_rtn, filter_factor, filter_n_equities, market_rtn, fees=None):
    df_filtered = df_full.reset_index()
    for n_equities, factor_to_use in zip(filter_n_equities[:-1], filter_factor[:-1]):
        eq_filter = select_best_equities_from_factor(df_filtered.set_index(['equity', 'date']), df_rtn, factor_to_use, n_equities)[0]
        df_filtered = pd.concat([df_filtered.loc[(df_filtered['date'] == date) & (df_filtered['equity'].isin(equities.values))
                                 ] for date, equities in eq_filter.iterrows()])
    return compute_univariate_strategy(df_filtered.set_index(['equity', 'date']), df_rtn, filter_factor[-1], filter_n_equities[-1], market_rtn, fees)

In [9]:
def evaluate_strategy(strategy_result, market_rtn, risk_free_rate):
    rtn, alpha, info_rate = strategy_result[1]['strategy_rtn'], strategy_result[1]['strategy_alpha'], strategy_result[2]

    rtn_mean = rtn.mean()
    std = rtn.std()
    downside_std = np.sqrt(np.sum((rtn[rtn < 0] - rtn.mean())**2) / (len(rtn) - 1))
    rtn_com = rtn.cumsum()
    alpha_com = alpha.cumsum()

    alpha_mean = alpha.mean()
    risk_adjusted = rtn_mean / std
    sharpe_ratio = (rtn - risk_free_rate).mean() / std
    beta = market_rtn.cov(rtn) / market_rtn.var()
    treynor_ratio = (rtn - risk_free_rate).mean()  / beta
    sortino_ratio = (rtn - risk_free_rate).mean() / downside_std

    stats = pd.Series(index=['info_ratio', 'rtn_avg', 'rtn_std', 'rtn_downside_std', 'rtn_tot', 'alpha_avg', 'alpha_tot', 'rtn_risk_adj', 'sharpe_ratio', 'beta', 'treynor_ratio', 'sortino_ratio'],
                     data=[info_rate, rtn_mean, std, downside_std, rtn_com.values[-1], alpha_mean, alpha_com.values[-1], risk_adjusted, sharpe_ratio, beta, treynor_ratio, sortino_ratio], name='stats')

    return stats, rtn, rtn_com, alpha, alpha_com


In [10]:
commission_fees = 0.002
num_equities_s = 15
# testing_factors = ['PE_RATIO', 'CURRENT_EV_TO_T12M_EBITDA', 'RSI_14D', 'VOLATILITY_90D', 'VOLATILITY_180D', 'EQY_REC_CONS', 'T12M_DIL_EPS_CONT_OPS', 'TRAIL_12M_EBITDA_PER_SHARE', 'NORMALIZED_ROE', 'CUR_MKT_CAP', 'OPERATING_ROIC', 'TANG_BOOK_VAL_PER_SH']
testing_factors = [
    '10_YEAR_MOVING_AVERAGE_PE', '5YR_AVG_RETURN_ON_EQUITY', 'CURRENT_EV_TO_T12M_EBITDA', 'CUR_MKT_CAP', 'EBITDA_MARGIN', 'EQY_DPS_NET_5YR_GROWTH', 'RSI_14D','OPERATING_ROIC',
    'EQY_REC_CONS', 'MOV_AVG_50D', 'NET_DEBT_PER_SHARE', 'PX_TO_BOOK_RATIO']

n_factor_to_use = 4
# testing_factors = ['PE_RATIO', 'EBITDA_MARGIN', 'PX_TO_BOOK_RATIO', 'NORMALIZED_ACCRUALS_CF_METHOD', 'RSI_14D', 'VOLATILITY_30D', 'CUR_MKT_CAP', 'OPERATING_ROIC']

# Applicazione delle strategie univariate su più fattori

In [11]:
# Calcolo equities e information ratio per ogni factor scelto
strs_res_stats = pd.DataFrame(columns=['info_ratio', 'rtn_avg', 'rtn_std', 'rtn_downside_std', 'rtn_tot', 'alpha_avg', 'alpha_tot', 'rtn_risk_adj', 'sharpe_ratio', 'beta', 'treynor_ratio', 'sortino_ratio'])
strs_res_rtn, strs_res_rtn_com, strs_res_rtn_alpha, strs_res_rtn_alpha_com = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
strs_res_port = {}

for factor in testing_factors:
    str_res = compute_univariate_strategy(df, df_log_rtn, factor, num_equities_s, benchmark_rtn, fees=commission_fees)
    strs_res_port[factor] = str_res[3]
    str_res = evaluate_strategy(str_res, benchmark_rtn, rfr)
    strs_res_stats.loc[factor] = str_res[0]
    strs_res_stats.at[factor, "factors_used"] = [factor]

    strs_res_rtn[factor] = str_res[1]
    strs_res_rtn_com[factor] = str_res[2]
    strs_res_rtn_alpha[factor] = str_res[3]
    strs_res_rtn_alpha_com[factor] = str_res[4]

strs_res_stats.iat[0, -1] = [strs_res_stats.iat[0, -1]]
best_factors = strs_res_stats.sort_values(by='info_ratio', ascending=False).head(n_factor_to_use).index.values

for factor in testing_factors:
    str_res = compute_univariate_strategy(df, df_log_rtn, factor, num_equities_s, benchmark_rtn)
    strs_res_port[factor + "_no-fees"] = str_res[3]
    str_res = evaluate_strategy(str_res, benchmark_rtn, rfr)
    strs_res_stats.loc[factor + "_no-fees"] = str_res[0]
    strs_res_stats.at[factor + "_no-fees", "factors_used"] = [factor]

    strs_res_rtn[factor + "_no-fees"] = str_res[1]
    strs_res_rtn_com[factor + "_no-fees"] = str_res[2]
    strs_res_rtn_alpha[factor + "_no-fees"] = str_res[3]
    strs_res_rtn_alpha_com[factor + "_no-fees"] = str_res[4]


# Applicazione delle strategie multivariate

In [12]:
seq_strategy = compute_sequential_screening(df, df_log_rtn, best_factors[:4][::-1], [400, 200, 100, 30], benchmark_rtn, fees=commission_fees)
zscore_strategy_simple = compute_zscore_strategy_simple(df, df_log_rtn, best_factors, num_equities_s, benchmark_rtn, fees=commission_fees)
zscore_strategy_weighted = compute_zscore_strategy_weighted(df, df_log_rtn, best_factors, num_equities_s, benchmark_rtn, fees=commission_fees)

F:\anaconda\envs\FMA\lib\site-packages\pandas\core\window\numba_.py:230: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  sub_result = numba_func(window, *args)
F:\anaconda\envs\FMA\lib\site-packages\pandas\core\window\numba_.py:234: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  result = np.where(min_periods_mask, result, np.nan)


# Calcolo delle statistiche per il benchmark

In [13]:
benchmark_rtn_com = benchmark_rtn.cumsum()
benchmark_stats = pd.Series(dtype='float64', name='benchmark_stats')

benchmark_stats['rtn_avg'] = benchmark_rtn.mean()
benchmark_stats['rtn_std'] = benchmark_rtn.std()
benchmark_stats['rtn_downside_std'] = np.sqrt(np.sum((benchmark_rtn[benchmark_rtn < 0] - benchmark_stats['rtn_std'])**2) / (len(benchmark_rtn) - 1))

benchmark_stats['rtn_tot'] = benchmark_rtn_com.values[-1]
benchmark_stats['rtn_risk_adj'] = benchmark_stats['rtn_avg'] / benchmark_stats['rtn_std']
benchmark_stats['sharpe_ratio'] = (benchmark_stats['rtn_avg'] - rfr).mean() / benchmark_stats['rtn_std']
# sortino ratio
benchmark_stats['sortino_ratio'] = (benchmark_stats['rtn_avg'] - rfr).mean() / benchmark_stats['rtn_downside_std']

strs_res_rtn['benchmark'] = benchmark_rtn
strs_res_rtn_com['benchmark'] = benchmark_rtn_com

benchmark_stats

rtn_avg             0.006537
rtn_std             0.052483
rtn_downside_std    0.066168
rtn_tot             0.719063
rtn_risk_adj        0.124553
sharpe_ratio       -0.643837
sortino_ratio      -0.510675
Name: benchmark_stats, dtype: float64

# Valutazione di tutte le strategie non univariate

In [14]:
for strategy, strategy_name, factor_used in zip([seq_strategy, zscore_strategy_simple, zscore_strategy_weighted],
                                                ["sequential", "zscore_simple", "zscore_weighted"],
                                                [best_factors]*3):
    strs_res_port[strategy_name] = strategy[3]
    str_res = evaluate_strategy(strategy, benchmark_rtn, rfr)
    strs_res_stats.loc[strategy_name] = str_res[0]
    strs_res_stats.at[strategy_name, "factors_used"] = factor_used

    strs_res_rtn[strategy_name] = str_res[1]
    strs_res_rtn_com[strategy_name] = str_res[2]
    strs_res_rtn_alpha[strategy_name] = str_res[3]
    strs_res_rtn_alpha_com[strategy_name] = str_res[4]

# Salvataggio dei risultati per il futuro plotting

In [15]:
benchmark_stats.to_csv(os.path.join(OUTPUT_PATH, "benchmark_stats.csv"))
strs_res_stats.to_csv(os.path.join(OUTPUT_PATH, "strs_res_stats.csv"))

strs_res_rtn.to_csv(os.path.join(OUTPUT_PATH, "rtn.csv"))
strs_res_rtn_com.to_csv(os.path.join(OUTPUT_PATH, "rtn_com.csv"))
strs_res_rtn_alpha.to_csv(os.path.join(OUTPUT_PATH, "alpha.csv"))
strs_res_rtn_alpha_com.to_csv(os.path.join(OUTPUT_PATH, "alpha_com.csv"))

with open(os.path.join(OUTPUT_PATH, "ports.pkl"), "wb") as f:
    pickle.dump(strs_res_port, f)